In [3]:
# 1.0 -  realiza a manipulação e limpeza de um conjunto de dados de óbitos a partir de um arquivo CSV, denominado Óbito.Inicialmente é feita a Leitura do arquivo CSV: Carrega um arquivo CSV contendo dados de óbitos em um DataFrame do Pandas.Após isso ocorre a Verificação de tipos de dados: Imprime o tipo de dados para cada coluna do DataFrame para inspeção inicial. Em seguida e realizada a conversão de colunas de data: Converte várias colunas relacionadas a datas (dt_registro, dt_obito, dt_nascimento, etc.) para o formato datetime, substituindo valores inválidos por NaT (Not a Time).Seguido da adição de novas colunas: Cria novas colunas no DataFrame e preenche todas com um valor padrão ("vazio").Para o tratamento de valores nulos,procede o preenchimento: Para colunas específicas, substitui strings vazias por None e, em seguida, preenche valores nulos com a string "sem informação".Para a conversão de colunas numéricas: Converte colunas especificadas para o tipo numérico, tratando erros e preenchendo valores nulos com 0, seguido da conversão para tipo inteiro.Para o tratamento de Formatação de colunas de data e hora: Converte colunas de datas e horas específicas para o formato datetime novamente, utilizando um formato específico para cada coluna.Preenchimento de valores ausentes em datas: Substitui valores vazios ou NaT nas colunas de data com um valor padrão (1900-01-01).Normalização de texto: Define uma função para normalizar o texto e corrigir problemas de codificação. Aplica essa função à coluna ds_explicacao_morte.Remoção de caracteres específicos: Remove o caractere / da coluna co_cid_causa_morte.Verificação final dos tipos de dados: Imprime os tipos de dados de colunas específicas após a manipulação.Salvamento do DataFrame atualizado: Escreve o DataFrame modificado para um novo arquivo CSV chamado dados_atualizados.csv.Esse código é focado na limpeza, padronização e preparação de dados para posterior análise ou processamento.

import pandas as pd
import re
df = pd.read_csv(r"C:\Users\anton\Desktop\ReapQuali\Obitos_Python\Dev\obitos.csv",delimiter=',')
df.head()

# Verificar o tipo de formato da coluna
for coluna in df.columns:
    print(f"Coluna: {coluna}, Tipo de dados: {df[coluna].dtypes}")
    
# Substituir valores inválidos por NaT (Not a Time) para facilitar a conversão
df['dt_registro'] = pd.to_datetime(df['dt_registro'], errors='coerce')
df['dt_obito'] = pd.to_datetime(df['dt_obito'], errors='coerce')
df['dt_nascimento'] = pd.to_datetime(df['dt_nascimento'], errors='coerce')
df['nu_tempo_aprox_morte'] = pd.to_datetime(df['nu_tempo_aprox_morte'], errors='coerce')
  
    
# Lista de nomes de novas colunas
novas_colunas = [
    "pb_nm_municipio",
    "pb_grs",
    "pb_regiao_saude",
    "pb_macro",
    "pb_nu_muni_ibge",
    "pb_ano_epidemiologico",
    "pb_semana_epidemiologica",
    "pb_te_sexo",
    "pb_te_raca",
    "fx_etaria",
    "unidades_saude"
]

# Adicionar as novas colunas ao DataFrame com valor padrão 'vazio'
for col_name in novas_colunas:
    df[col_name] = " "

# Lista de colunas com valores nulos preenchidas 
colunas_para_preencher = ['nu_cep_ocorrencia', 'ds_complemento_ocorrencia', 'no_medico','no_logradouro_ocorrencia']

# Substituir strings vazias por None para padronizar
df[colunas_para_preencher] = df[colunas_para_preencher].replace('', None)

# Preencher valores nulos com "sem informação"
df[colunas_para_preencher] = df[colunas_para_preencher].fillna("sem informacao")

# Lista de colunas float para converter
float_columns = [
    'co_municipio_ibge_cartorio', 'co_cartorio', 'nu_registro', 'co_naturalidade',
    'nu_idade', 'tp_idade', 'tp_raca_cor', 'tp_estado_civil', 'tp_escolaridade',
    'co_ocupacao_falecido', 'co_municipio_ibge_residencia', 'co_regiao_residencia',
    'co_pais_residencia','co_estabelecimento','co_municipio_ibge_estab','co_bairro_ocorrencia','nu_bairro_ocorrencia','co_logradouro_ocorrencia','nu_logradouro_ocorrencia','st_assistencia_medica','st_exame_complementar','st_cirurgia','st_necropsia','nu_crm','tp_medico_assina','st_morte_mulher','tp_morte_puerperio',
]

for col in float_columns:
    # Converter para numérico e tratar erros
    df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Preencher valores NaN com 0 (ou outro valor apropriado)
    df[col] = df[col].fillna(0)
    
    # Converter para inteiro
    df[col] = df[col].astype(int)

# Converter colunas de data e hora para datetime
df['dt_registro'] = pd.to_datetime(df['dt_registro'], format='%d/%m/%Y %H:%M:%S', errors='coerce')
df['dt_obito'] = pd.to_datetime(df['dt_obito'], format='%d/%m/%Y', errors='coerce')
df['hr_obito'] = pd.to_datetime(df['hr_obito'], format='%H:%M:%S', errors='coerce').dt.time
df['dt_nascimento'] = pd.to_datetime(df['dt_nascimento'], format='%d/%m/%Y', errors='coerce')
df['dt_atestado'] = pd.to_datetime(df['dt_atestado'], format='%d/%m/%Y', errors='coerce')
df['dt_investigacao'] = pd.to_datetime(df['dt_atestado'], format='%d/%m/%Y', errors='coerce')

# Define o valor padrão para datas ausentes
valor_padrao = pd.Timestamp('1900-01-01')

# Lista de colunas de data e hora
colunas_data = ['dt_registro','dt_obito','dt_nascimento']

for col in colunas_data:
    # Substituir valores vazios por NaT
    df[col] = df[col].replace('', pd.NaT)
    
    # Preencher valores NaT com o valor padrão
    if col == 'hr_obito':
        # Para horas, não faz sentido usar um valor de data. Usar um valor de tempo padrão pode ser melhor.
        df[col] = df[col].fillna(pd.Timestamp('1900-01-01').time())
    else:
        df[col] = df[col].fillna(valor_padrao)
        
# Função para normalizar e substituir o texto
def normalizar_e_substituir(texto):
    if isinstance(texto, str):
        # Normalizar o texto, removendo possíveis problemas de codificação
        texto = texto.encode('latin1', errors='ignore').decode('utf-8', errors='ignore')
        # Substituir 'PrincÃ­pio Geral' por 'Causa Principal'
        return texto.replace('Princpio Geral', 'Causa Principal')
    return texto

# Aplicar a função para normalizar e substituir o texto na coluna ds_explicacao_morte
df['ds_explicacao_morte'] = df['ds_explicacao_morte'].apply(normalizar_e_substituir)

# Remover o caractere '/' da coluna 'co_cid_causa_morte'
df['co_cid_causa_morte'] = df['co_cid_causa_morte'].str.replace('/', '', regex=False)

# Verificar o tipo de formato da coluna individualmente
print(df['dt_registro'].dtypes)
print(df['co_municipio_ibge_cartorio'].dtypes) 
print(df['dt_nascimento'].dtypes) 
print(df['dt_atestado'].dtypes) 
print(df['co_estabelecimento'].dtypes) 
print(df['ds_explicacao_morte'].dtypes) 
print(df['co_cid_causa_morte'])

# Salvar o DataFrame atualizado em um novo arquivo CSV
df.to_csv('dados_atualizados.csv', index=False)




C:\Users\anton\AppData\Local\Temp\ipykernel_3428\1754331217.py:4: DtypeWarning: Columns (60,61,62,63,64,74,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\anton\Desktop\ReapQuali\Obitos_Python\Teste\obitos.csv",delimiter=',')


Coluna: Unnamed: 0, Tipo de dados: int64
Coluna: co_municipio_ibge, Tipo de dados: int64
Coluna: nu_micro, Tipo de dados: int64
Coluna: co_seq_do, Tipo de dados: int64
Coluna: tp_nivel, Tipo de dados: int64
Coluna: nu_do, Tipo de dados: int64
Coluna: co_municipio_ibge_cartorio, Tipo de dados: float64
Coluna: co_cartorio, Tipo de dados: float64
Coluna: nu_registro, Tipo de dados: float64
Coluna: dt_registro, Tipo de dados: object
Coluna: tp_obito, Tipo de dados: int64
Coluna: dt_obito, Tipo de dados: object
Coluna: hr_obito, Tipo de dados: object
Coluna: co_naturalidade, Tipo de dados: float64
Coluna: dt_nascimento, Tipo de dados: object
Coluna: nu_idade, Tipo de dados: float64
Coluna: sg_sexo, Tipo de dados: object
Coluna: tp_idade, Tipo de dados: float64
Coluna: tp_raca_cor, Tipo de dados: float64
Coluna: tp_estado_civil, Tipo de dados: float64
Coluna: tp_escolaridade, Tipo de dados: float64
Coluna: co_ocupacao_falecido, Tipo de dados: float64
Coluna: co_municipio_ibge_residencia, Tip

C:\Users\anton\AppData\Local\Temp\ipykernel_3428\1754331217.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['nu_tempo_aprox_morte'] = pd.to_datetime(df['nu_tempo_aprox_morte'], errors='coerce')


datetime64[ns]
int64
datetime64[ns]
datetime64[ns]
int64
object
0                      I219 
1            B572 I469 I219 
2                      I219 
3        I219 I251 I709 I10 
4                      I219 
                ...         
50181                I10I219
50182        I219J81J189I509
50183    R570I219I251I10E149
50184       I461I219I10 F175
50185        I219I119I10E119
Name: co_cid_causa_morte, Length: 50186, dtype: object
